In [6]:
from futu import *
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

ret, data = quote_ctx.get_market_snapshot(['HK.09988'])
if ret == RET_OK:
    last_price = data['last_price'][0]  # 最新价格
    print(last_price)

else:
    print('can not get latest price', data)
quote_ctx.close() # 结束后记得关闭当条连接，防止连接条数用尽


2023-11-11 09:31:52,086 | 27393 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1063, host=127.0.0.1, port=11111, user_id=28749530
79.6
2023-11-11 09:31:52,161 | 27393 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=1063


2023-11-11 09:31:55,691 | 27393 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1064, host=127.0.0.1, port=11111, user_id=28749530
2023-11-11 09:31:55,692 | 27393 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1065, host=127.0.0.1, port=11111, user_id=28749530


In [7]:
from futu import *
trd_ctx = OpenSecTradeContext(filter_trdmarket=TrdMarket.HK, host='127.0.0.1', port=11111, security_firm=SecurityFirm.FUTUSECURITIES)
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

ret, data = trd_ctx.acctradinginfo_query(trd_env=TrdEnv.SIMULATE, order_type=OrderType.NORMAL, code='HK.09988', price=88)
if ret == RET_OK:
    print(data)
    print(data['max_cash_and_margin_buy'][0])  # Get maximum quantity that can be bought on margin
else:
    print('acctradinginfo_query error: ', data)
trd_ctx.close()  # Close the current connection


2023-11-11 09:31:57,459 | 27393 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1066, host=127.0.0.1, port=11111, user_id=28749530
2023-11-11 09:31:57,469 | 27393 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1067, host=127.0.0.1, port=11111, user_id=28749530
   max_cash_buy  max_cash_and_margin_buy  max_position_sell max_sell_short  \
0         500.0                    500.0             8000.0            N/A   

  max_buy_back long_required_im short_required_im  
0          N/A              N/A               N/A  
500.0
2023-11-11 09:31:57,545 | 27393 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=1066


In [11]:
from futu import *
from utilts.encrypt import decrypt

PASSWORD = os.environ.get('FUTU_PASSWORD')
############################ 全局变量设置 ############################
FUTUOPEND_ADDRESS = '127.0.0.1'  # Futu OpenD 监听地址
FUTUOPEND_PORT = 11111  # Futu OpenD 监听端口

TRADING_ENVIRONMENT = TrdEnv.SIMULATE  # 交易环境：真实 / 模拟
TRADING_MARKET = TrdMarket.HK  # 交易市场权限，用于筛选对应交易市场权限的账户
TRADING_PWD = decrypt(PASSWORD)  # 交易密码，用于解锁交易
TRADING_PERIOD = KLType.K_1M  # 信号 K 线周期
TRADING_SECURITY = 'HK.09988'  # 交易标的

quote_context = OpenQuoteContext(host=FUTUOPEND_ADDRESS, port=FUTUOPEND_PORT)  # 行情对象
trade_context = OpenSecTradeContext(filter_trdmarket=TRADING_MARKET, host=FUTUOPEND_ADDRESS, port=FUTUOPEND_PORT, security_firm=SecurityFirm.FUTUSECURITIES)  # 交易对象，根据交易品种修改交易对象类型
# 解锁交易
def unlock_trade():
    if TRADING_ENVIRONMENT == TrdEnv.REAL:
        ret, data = trade_context.unlock_trade(TRADING_PWD)
        if ret != RET_OK:
            print('解锁交易失败：', data)
            return False
        print('解锁交易成功！')
    return True

############################ 填充以下函数来完成您的策略 ############################
# 策略启动时运行一次，用于初始化策略
def on_init():
    # 解锁交易（如果是模拟交易则不需要解锁）
    if not unlock_trade():
        return False
    print('************  策略开始运行 ***********')
    return True

on_init()

253908
2023-11-11 09:33:42,616 | 27393 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1074, host=127.0.0.1, port=11111, user_id=28749530
2023-11-11 09:33:42,619 | 27393 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1075, host=127.0.0.1, port=11111, user_id=28749530
************  策略开始运行 ***********


True